In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib import pyplot

### Importing the dataset using the url

In [ ]:
path="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/auto.csv"
df=pd.read_csv(path,header=None,names=["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"])
df.head(10)

### Check number of rows and columns

In [ ]:
df.shape

### Steps for working with missing data:

<ol>
    <li>Identify missing data</li>
    <li>Deal with missing data</li>
    <li>Correct data format</li>
</ol>

# <u>Identify and handle missing values</u>


### Replacing '?' with NaN

In [ ]:
df=df.replace('?',np.NaN)
df.head(10)

### Checking number of null values in each column

In [ ]:
missing=df.isnull().sum()
missing=missing[missing>0]
missing

# <u>Deal with missing data</u>
<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li><br>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


Whole columns should be dropped only if most entries in the column are empty. In our dataset, none of the columns are empty enough to drop entirely.
We have some freedom in choosing which method to replace data; however, some methods may seem more reasonable than others. We will apply each method to many different columns:

<b>Replace by mean:</b>

<ul>
    <li>"normalized-losses": 41 missing data, replace them with mean</li>
    <li>"stroke": 4 missing data, replace them with mean</li>
    <li>"bore": 4 missing data, replace them with mean</li>
    <li>"horsepower": 2 missing data, replace them with mean</li>
    <li>"peak-rpm": 2 missing data, replace them with mean</li>
</ul>

<b>Replace by frequency:</b>

<ul>
    <li>"num-of-doors": 2 missing data, replace them with "four". 
        <ul>
            <li>Reason: 84% sedans is four doors. Since four doors is most frequent, it is most likely to occur</li>
        </ul>
    </li>
</ul>

<b>Drop the whole row:</b>

<ul>
    <li>"price": 4 missing data, simply delete the whole row
        <ul>
            <li>Reason: price is what we want to predict. Any data entry without price data cannot be used for prediction; therefore any row now without price data is not useful to us</li>
        </ul>
    </li>
</ul>


### Dropping rows where price is null

In [ ]:
df=df.dropna(subset=['price'],axis=0)
df.head(10)

### Replacing NaN in "normalized-losses" with its mean value

In [ ]:
df= df.fillna({'normalized-losses':df['normalized-losses'].astype('float').mean()})
df.head(10)

### Replacing NaN in "stroke" with its mean value


In [ ]:
df= df.fillna({'stroke':df['stroke'].astype('float').mean()})
df.head(10)

### Replacing NaN in "bore" with its mean value


In [ ]:
df= df.fillna({'bore':df['bore'].astype('float').mean()})
df.head(10)

### Replacing NaN in "horsepower" with its mean value


In [ ]:
df= df.fillna({'horsepower':df['horsepower'].astype('float').mean()})
df.head(10)

### Replacing NaN in "normalized-losses" with its mean value


In [ ]:
df= df.fillna({'peak-rpm':df['peak-rpm'].astype('float').mean()})
df.head(10)

### Replacing NaN in "num-of-doors" with its mode value


In [ ]:
df=df.fillna({'num-of-doors':df['num-of-doors'].mode()[0]})
df.head(10)


To see which values are present in a particular column, we can use the ".value_counts()" method:


In [ ]:
df['num-of-doors'].value_counts()

reset index

In [ ]:
df.reset_index(drop=True,inplace=True)
df.head()

# <u>Correct Data Format</u>

#### Lets check the data type of each column

In [ ]:
df.dtypes

#### Convert data types to proper format


In [ ]:
df[["bore", "stroke","peak-rpm","price"]] = df[["bore", "stroke","peak-rpm","price"]].astype("float")
df[["normalized-losses","horsepower"]] = df[["normalized-losses","horsepower"]].astype("int")

#### After the conversion

In [ ]:
df.dtypes

# <u>Data Standardization</u>

### Converting mpg to L/100Km in city-mpg and highway-mpg columns

In [ ]:
df['city-L/100km'] = 235/df["city-mpg"]
df['highway-L/100km'] = 235/df["highway-mpg"]
df.head()


# <u>Data Normalization</u>

### scale length,width and height column

In [ ]:
df['length']=df['length']/df['length'].max()
df['width']=df['width']/df['width'].max()
df['height']=df['height']/df['height'].max()
df.head()


# <u>Binning</u>


<p>In our dataset, "horsepower" is a real valued variable ranging from 48 to 288 and it has 59 unique values. What if we only care about the price difference between cars with high horsepower, medium horsepower, and little horsepower (3 types)? Can we rearrange them into three ‘bins' to simplify analysis? </p>

### Distribution of horsepower using histogram

In [ ]:
plt.pyplot.hist(df["horsepower"])

# set x/y labels and plot title
plt.pyplot.xlabel("horsepower")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower bins")

### Binning the horsepower column 


In [ ]:
power_bins = np.linspace(df['horsepower'].min(),df['horsepower'].max(),4)
group_names=["Low","medium","high"]
df['horsepower-range'] = pd.cut(df.horsepower,power_bins,labels=group_names,include_lowest=True)
df.head() 


#### lets see the number of vehicles in each bin

In [ ]:
df["horsepower-range"].value_counts()

### Visualizing distribution of each bin 

In [ ]:
pyplot.bar(group_names, df["horsepower-range"].value_counts())

#assigning labels
plt.pyplot.xlabel("horse power range")
plt.pyplot.ylabel("count")
plt.pyplot.title("horsepower Levels")

### Binning the price column 

In [ ]:
price_bins = np.linspace(min(df.price),max(df.price),4)
group_names=["Low","medium","high"]
df["price-range"]=pd.cut(df.price,price_bins,labels=group_names,include_lowest=True)
df.head(10)

In [ ]:
plt.pyplot.bar(group_names,df["price-range"].value_counts())

#labels
plt.pyplot.xlabel("Price range")
plt.pyplot.ylabel("count")
plt.pyplot.title("price levels")

# <u>Indicator Variable (or Dummy Variable)</u>
<b>What is an indicator variable?</b>
<p>
    An indicator variable (or dummy variable) is a numerical variable used to label categories. They are called 'dummies' because the numbers themselves don't have inherent meaning. 
</p>

<b>Why we use indicator variables?</b>

<p>
    We use indicator variables so we can use categorical variables for regression analysis in the later modules.
</p>
<b>Example</b>
<p>
    We see the column "fuel-type" has two unique values: "gas" or "diesel". Regression doesn't understand words, only numbers. To use this attribute in regression analysis, we convert "fuel-type" to indicator variables.
</p>

<p>
    We will use pandas' method 'get_dummies' to assign numerical values to different categories of fuel type. 
</p>

In [ ]:
dummy_fuel_var=pd.get_dummies(df['fuel-type'])
dummy_fuel_var.head()

changing the column names for clarity

In [ ]:
dummy_fuel_var.rename(columns={'gas':'fuel-type-gas','diesel':'fuel-type-diesel'},inplace=True)
dummy_fuel_var.head()


##### merge data frame "df" and "dummy_fuel_var" 


In [79]:
df=pd.concat([df,dummy_fuel_var],axis=1)

#drop fuel type from original dataframe df
df = df.drop('fuel-type',axis=1)

similarly we will create dummy variable for aspiration column

In [ ]:
df.aspiration.value_counts()

In [ ]:
dummy_aspiration_var=pd.get_dummies(df['aspiration'])
dummy_aspiration_var.head()

##### renaming the column for clarity

In [ ]:
dummy_aspiration_var.rename(columns={'std':'aspiration-std','turbo':'aspiration-turbo'},inplace=True)
dummy_aspiration_var.head()

##### merge data frame "df" and "dummy_aspiration_var" 


In [81]:
df=pd.concat([df,dummy_aspiration_var],axis=1)

#droping the aspiration column from original dataframe df
df = df.drop(['aspiration'],axis=1)

In [82]:
df.head()

,symboling,normalized-losses,make,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,...,highway-mpg,price,city-L/100km,highway-L/100km,horsepower-range,price-range,fuel-type-diesel,fuel-type-gas,aspiration-std,aspiration-turbo
0,3,122,alfa-romero,two,convertible,rwd,front,88.6,0.811148,0.890278,...,27,13495.0,11.190476,8.703704,Low,Low,0,1,1,0
1,3,122,alfa-romero,two,convertible,rwd,front,88.6,0.811148,0.890278,...,27,16500.0,11.190476,8.703704,Low,Low,0,1,1,0
2,1,122,alfa-romero,two,hatchback,rwd,front,94.5,0.822681,0.909722,...,26,16500.0,12.368421,9.038462,medium,Low,0,1,1,0
3,2,164,audi,four,sedan,fwd,front,99.8,0.848630,0.919444,...,30,13950.0,9.791667,7.833333,Low,Low,0,1,1,0
4,2,164,audi,four,sedan,4wd,front,99.4,0.848630,0.922222,...,22,17450.0,13.055556,10.681818,Low,Low,0,1,1,0


# <u>Saving the cleaned and processed dataframe</u>

In [83]:
df.to_csv('cleaned_automobile.csv')